In [ ]:
# analysis notebook 
# then can correlate energy use with cost, 
# as well as tornado diagrams for energy use and cost -> subplots
# can compare individual schemes to see which give greatest e savings / cost 
# can pair schemes to maximize..

# see opt_fun.py for how these are calculated 
# see the individual files for the data 

In [12]:
import pandas as pd
import os
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [64]:
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls

username = "jnwagwu"
api_key = "119wNvkoTSGwHIxORQ85"

chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

# Import Data

In [58]:
group_models = ["shgc_b1", "roof_b1", "wall_b1", "floor_b1"]

In [19]:
root_path = os.getcwd()
opt_models_path= os.path.join(root_path,"opt_models")


In [20]:
def get_group_csv(group_model):
    group_model_path = os.path.join(opt_models_path, group_model )
    csv_path = os.path.join(group_model_path, "data.csv")
    df = pd.read_csv(csv_path, index_col="Unnamed: 0")
    return df

In [59]:
group_data = {model: get_group_csv(model) for model in group_models}
group_data.keys()

dict_keys(['shgc_b1', 'roof_b1', 'wall_b1', 'floor_b1'])

In [66]:
group_data["floor_b1"]

,elect,hot,chill,elect_c,hot_c,chill_c,total,total_c,vals,perc_change
-98,104929.749726,21627.681852,63434.691672,19472.53,2951.87,7575.69,189992.123251,30000.09,0.003550,-98
-50,106310.963727,19827.866663,71874.795428,19729.25,2706.22,8583.64,198013.625818,31019.11,0.088747,-50
-25,106639.557240,19305.993586,75128.700255,19772.62,2634.99,8972.24,201074.251080,31379.85,0.133120,-25
0,106895.375499,18940.063394,77717.985229,19805.56,2585.05,9281.47,203553.424123,31672.08,0.177494,0
25,107125.739938,18664.772540,79705.967296,19840.67,2547.47,9518.88,205496.479775,31907.02,0.221867,25
50,107314.030177,18447.781802,81338.409648,19871.44,2517.86,9713.84,207100.221628,32103.14,0.266241,50
98,107586.794473,18132.790345,83775.962582,19914.05,2474.87,10004.94,209495.547400,32393.86,0.351438,98


,total,vals
total,1.000000,0.966771
vals,0.966771,1.000000


# Examine Data

In [60]:
fig = go.Figure()

for k in group_data.keys(): 

    # y = group_data[k]["total"] - group_data[k].loc[0]["total"]
    y = group_data[k]["total"]

    x = group_data[k]["vals"]
    fig.add_trace(
                go.Scatter(x=x, y=y, name=k)
                )
fig.update_layout(xaxis_title=f"Vals (R-Val or SHGC)",
            yaxis_title="Energy (kWh)",)

py.plot(fig, filename="g1", auto_open=False)

In [61]:
fig = go.Figure()

for k in group_data.keys(): 
    # y = group_data[k]["total"] - group_data[k].loc[0]["total"]
    y = group_data[k]["total"]

    x = group_data[k]["perc_change"]
    fig.add_trace(
                go.Scatter(x=x, y=y, name=k)
                )
fig.update_layout(xaxis_title=f"Percent Change from Original",
            yaxis_title="Energy (kWh)",)

fig.show()

py.plot(fig, filename="g2", auto_open=False)


In [62]:
# plot scatter plot energy savings vs cost difference  
fig = go.Figure()

for k in group_data.keys():
        x = group_data[k]["total_c"] - group_data[k].loc[0]["total_c"]
        y = group_data[k]["total"] - group_data[k].loc[0]["total"]
        
        fig.add_trace(

                go.Scatter(x=x, y=y, name=k)
                )
fig.update_layout(xaxis_title=f"Cost Delta [$]",
            yaxis_title="Energy Delta [kWh]",)

fig.show()

py.plot(fig, filename="g3", auto_open=False)

# linear relatuonship as expected between cost and energy use 


In [63]:
# plot scatter plot energy savings vs cost difference  
fig = go.Figure()

for k in group_data.keys():
        x = group_data[k]["perc_change"]
        y = group_data[k]["total"] - group_data[k].loc[0]["total"]
        
        fig.add_trace(

                go.Scatter(x=x, y=y, name=k)
                )
fig.update_layout(xaxis_title=f"Percent Change",
            yaxis_title="Energy Delta [kWh]",)

fig.show()

py.plot(fig, filename="g4", auto_open=False)

# Correlations

In [75]:
# calculate correlations 
fig = go.Figure()

corr_data = {}
for k in group_data.keys():
    corr_df = group_data[k][["total", "vals"]].corr()
    corr_data[k] = corr_df.loc["vals"]["total"]

fig.add_trace(go.Bar(x=list(corr_data.keys()), y=list(corr_data.values())))
fig.update_layout(xaxis_title=f"Groups",
            yaxis_title="Correlation between Value and Energy Use",)

fig.show()

py.plot(fig, filename="g5", auto_open=False)

'https://plotly.com/~jnwagwu/5/'

most effective to change floor and shgc to increase / decrease energy use since they have a higher correlation with energy.. however fell as though this is biased because of the low starting values of shgc and the floor r value. 